### Import libraries and create DataFrame from csv 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import Ridge
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error

# Function to read, preprocess, and aggregate full season data
def read_data():
    # Read the full season data
    data = pd.read_excel("WR_Data_2024.xlsx")

    # Preprocess full season data, drop irrelevant columns and convert data types
    data.drop(['FL', 'ROST'], axis=1, inplace=True)
    convert = data.select_dtypes('object').columns.difference(['Player'])
    data[convert] = data[convert].apply(lambda x: pd.to_numeric(x.str.replace(',', ''), errors='coerce')).fillna(0)
    data['Rank'] = data['Rank'].astype('Int64')

    return data

# Load the full season data
df = read_data()

# Recalculate 'Rank' based on 'FPTS' in descending order
df['Rank'] = df['FPTS'].rank(ascending=False, method='min').astype('int')

# Sort the DataFrame by 'Rank'
df = df.sort_values(by='Rank')
df.head(20)

,Rank,Player,REC,TGT,YDS,Y/R,LG,20+,TD,ATT,YDS.1,TD.1,G,FPTS,FPTS/G
0,1,Malik Nabers (NYG),23,37,271,11.8,28,6,3,2,4,0,3,68.5,22.8
1,2,Rashee Rice (KC),24,29,288,12.0,44,3,2,1,1,0,3,64.9,21.6
2,3,Chris Godwin (TB),21,25,253,12.0,41,2,3,0,0,0,3,64.3,21.4
3,4,Jauan Jennings (SF),18,21,276,15.3,34,5,3,0,0,0,3,63.6,21.2
4,5,Justin Jefferson (MIN),14,21,273,19.5,97,4,3,0,0,0,3,59.3,19.8
5,6,Nico Collins (HOU),18,28,338,18.8,55,5,1,0,0,0,3,57.8,19.3
6,7,DK Metcalf (SEA),17,24,262,15.4,71,3,2,0,0,0,3,55.2,18.4
7,8,DeVonta Smith (PHI),21,28,239,11.4,25,2,1,0,0,0,3,50.9,17.0
8,9,Jayden Reed (GB),10,14,197,19.7,70,4,1,5,89,1,3,50.6,16.9
9,10,Stefon Diggs (HOU),20,24,164,8.2,21,1,2,1,6,0,3,49.5,16.5


### Convert relevant stats to a per-game basis and create the final stats for analysis

In [2]:
# Define columns for per-game calculation
per_game = list(df.columns[2:])

# Define columns to exclude from the per-game calculation
exclude_per_game = ['Y/R', 'LG', 'G', 'FPTS', 'FPTS/G']

# Create columns and convert stats to a per-game basis for the defined columns, not those in the exclude list
for col in per_game:
    if col not in exclude_per_game:
        df[col + '/game'] = (df[col] / df['G']).round(1)

# Define final columns to be used for analysis, combining excluded and new per-game columns
final_columns = exclude_per_game + [col + '/game' for col in per_game if col not in exclude_per_game]

# Display final columns with 'Rank' and 'Player'
df[['Rank', 'Player'] + final_columns].head(10)

,Rank,Player,Y/R,LG,G,FPTS,FPTS/G,REC/game,TGT/game,YDS/game,20+/game,TD/game,ATT/game,YDS.1/game,TD.1/game
0,1,Malik Nabers (NYG),11.8,28,3,68.5,22.8,7.7,12.3,90.3,2.0,1.0,0.7,1.3,0.0
1,2,Rashee Rice (KC),12.0,44,3,64.9,21.6,8.0,9.7,96.0,1.0,0.7,0.3,0.3,0.0
2,3,Chris Godwin (TB),12.0,41,3,64.3,21.4,7.0,8.3,84.3,0.7,1.0,0.0,0.0,0.0
3,4,Jauan Jennings (SF),15.3,34,3,63.6,21.2,6.0,7.0,92.0,1.7,1.0,0.0,0.0,0.0
4,5,Justin Jefferson (MIN),19.5,97,3,59.3,19.8,4.7,7.0,91.0,1.3,1.0,0.0,0.0,0.0
5,6,Nico Collins (HOU),18.8,55,3,57.8,19.3,6.0,9.3,112.7,1.7,0.3,0.0,0.0,0.0
6,7,DK Metcalf (SEA),15.4,71,3,55.2,18.4,5.7,8.0,87.3,1.0,0.7,0.0,0.0,0.0
7,8,DeVonta Smith (PHI),11.4,25,3,50.9,17.0,7.0,9.3,79.7,0.7,0.3,0.0,0.0,0.0
8,9,Jayden Reed (GB),19.7,70,3,50.6,16.9,3.3,4.7,65.7,1.3,0.3,1.7,29.7,0.3
9,10,Stefon Diggs (HOU),8.2,21,3,49.5,16.5,6.7,8.0,54.7,0.3,0.7,0.3,2.0,0.0


### Calculate the correlations for the final stats across different conditions

In [3]:
# Define columns and columns to exclude for correlation calculation, excluding FPTS and FPTS/G
exclude_corr = ['FPTS/G', 'FPTS', 'G']
corr_columns = [col for col in final_columns if col not in exclude_corr]

# Define a function to calculate correlations
def compute_correlations(dataframe, corr_columns):
    return dataframe[corr_columns].corrwith(dataframe['FPTS/G'])

# Compute correlations for various conditions
corr_all = compute_correlations(df, corr_columns)
corr_nonzero = compute_correlations(df[df['FPTS/G'] > 0], corr_columns)
corr_top50 = compute_correlations(df[df['Rank'] <= 50], corr_columns)
corr_top25 = compute_correlations(df[df['Rank'] <= 25], corr_columns)

# Compile all correlations into a DataFrame for comparison
df_corr = pd.DataFrame({
    'All Players': corr_all,
    'FPTS > 0': corr_nonzero,
    'Top 50 Players': corr_top50,
    'Top 25 Players': corr_top25
})

# Calculate the average correlation across the four conditions, adding 'Average' column to DataFrame
df_corr['Correlation'] = df_corr.mean(axis=1)

# Display the correlation DataFrame
df_corr.round(2)

,All Players,FPTS > 0,Top 50 Players,Top 25 Players,Correlation
Y/R,0.68,0.31,0.11,-0.15,0.24
LG,0.81,0.64,0.19,-0.03,0.40
REC/game,0.92,0.89,0.70,0.70,0.80
TGT/game,0.89,0.85,0.56,0.56,0.71
YDS/game,0.96,0.95,0.85,0.79,0.89
20+/game,0.80,0.77,0.61,0.63,0.70
TD/game,0.77,0.75,0.56,0.34,0.61
ATT/game,0.27,0.23,0.19,0.03,0.18
YDS.1/game,0.25,0.20,0.10,-0.11,0.11
TD.1/game,0.22,0.21,0.09,0.11,0.16


### Assign the weights for the final stats

In [4]:
# Calculate R^2 for the 'Average' correlation, adding 'R^2' column to DataFrame
df_corr['R^2'] = df_corr['Correlation'] ** 2

# Select stats with R^2 above a threshold for higher weight
high_weight_threshold = 0.5
specific_stats = df_corr[df_corr['R^2'] > high_weight_threshold].index.tolist()

# Select stats with R^2 below a threshold to exclude from final score
exclude_threshold = 0.1
exclude_stats = df_corr[df_corr['R^2'] < exclude_threshold].index.tolist()

# Define the calculation to assign weights
def weight_calc(row, specific_stats):
    if row.name in specific_stats:
        return 1 + row['R^2'] * 2 # Assign higher weight to specific stats
    else:
        return 1 + row['R^2'] # Assign weight to all other stats
    
# Assign weights based on the given criteria, adding 'Weight' column to DataFrame
df_corr['Weight'] = df_corr.apply(weight_calc, specific_stats=specific_stats, axis=1)

# Normalize weights
df_corr['Weight'] = df_corr['Weight'] / df_corr['Weight'].sum()

# Display the new columns in the DataFrame
df_corr[['Correlation', 'R^2', 'Weight']].round(2)

,Correlation,R^2,Weight
Y/R,0.24,0.06,0.07
LG,0.40,0.16,0.08
REC/game,0.80,0.64,0.15
TGT/game,0.71,0.51,0.13
YDS/game,0.89,0.79,0.17
20+/game,0.70,0.49,0.10
TD/game,0.61,0.37,0.09
ATT/game,0.18,0.03,0.07
YDS.1/game,0.11,0.01,0.07
TD.1/game,0.16,0.02,0.07


### Multiply the assigned weights to the final stats and calculate the score

In [5]:
# Multiply each relevant column by its corresponding weight
for col in corr_columns:
    if col not in exclude_stats:
        weight = df_corr.loc[col, 'Weight']
        df[col + '_weighted'] = (df[col] * weight).round(1)

# Extract the weighted columns
weight_columns = [col + '_weighted' for col in corr_columns if col not in exclude_stats]

# Display the new weighted stats columns
df_weight = df[['Rank', 'Player', 'FPTS/G'] + weight_columns]
df_weight.sort_values(by=['Rank'], ascending=True).head(10)

# Define select columns to be used for the average weighted score
avg = weight_columns + (['FPTS/G'] * 2)  # Giving higher weight to FPTS/G

# Calculate the average weighted score for the select columns
df['Score'] = df[avg].mean(axis=1).round(2)

# Normalize the scores to be out of 10
scaler = MinMaxScaler(feature_range=(0, 10))
df['Score'] = scaler.fit_transform(df[['Score']])

df_weight.head(10)

,Rank,Player,FPTS/G,LG_weighted,REC/game_weighted,TGT/game_weighted,YDS/game_weighted,20+/game_weighted,TD/game_weighted
0,1,Malik Nabers (NYG),22.8,2.2,1.2,1.7,15.5,0.2,0.1
1,2,Rashee Rice (KC),21.6,3.4,1.2,1.3,16.5,0.1,0.1
2,3,Chris Godwin (TB),21.4,3.2,1.1,1.1,14.5,0.1,0.1
3,4,Jauan Jennings (SF),21.2,2.6,0.9,0.9,15.8,0.2,0.1
4,5,Justin Jefferson (MIN),19.8,7.5,0.7,0.9,15.6,0.1,0.1
5,6,Nico Collins (HOU),19.3,4.3,0.9,1.3,19.4,0.2,0.0
6,7,DK Metcalf (SEA),18.4,5.5,0.9,1.1,15.0,0.1,0.1
7,8,DeVonta Smith (PHI),17.0,1.9,1.1,1.3,13.7,0.1,0.0
8,9,Jayden Reed (GB),16.9,5.4,0.5,0.6,11.3,0.1,0.0
9,10,Stefon Diggs (HOU),16.5,1.6,1.0,1.1,9.4,0.0,0.1


### Model training

In [6]:
# Prepare features and target for model training
X = df[weight_columns]
y = df['Score']

# Handle missing values by imputing with mean
imputer = SimpleImputer(strategy='mean')
X = imputer.fit_transform(X)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test, train_idx, test_idx = train_test_split(X, y, df.index, test_size=0.2, random_state=42)

# Initialize Ridge regression model
ridge_model = Ridge()

# Cross-validation to evaluate the model
cv_scores = cross_val_score(ridge_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
print(f'Cross-Validation MSE: {-cv_scores.mean()}')

# Train the model
ridge_model.fit(X_train, y_train)

# Predict and evaluate on the test set
y_pred = ridge_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f'Test Set MSE: {mse}')

# Incorporate Random Forest as an ensemble method
rf_model = RandomForestRegressor(n_estimators=100, random_state=42)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_mse = mean_squared_error(y_test, rf_pred)
print(f'Random Forest Test Set MSE: {rf_mse}')

# Average predictions from both models for final score
final_pred = (y_pred + rf_pred) / 2

# Create a DataFrame for the test set predictions
test_results = pd.DataFrame({'Final_Score': final_pred}, index=test_idx)

# Merge the test set predictions back into the original DataFrame
df = df.merge(test_results, how='left', left_index=True, right_index=True)

# Fill NaN values in 'Final_Score' column with the original 'Score' to handle missing indices
df['Final_Score'].fillna(df['Score'], inplace=True)

# Normalize final scores to be out of 10
df['Final_Score'] = scaler.fit_transform(df[['Final_Score']]).round(2)

# Rank the final scores
df['Final Rank'] = df['Final_Score'].rank(method='first', ascending=False).astype(int)

# Calculate the variance in ranking both ranks
df['Variance'] = df['Rank'] - df['Final Rank']

Cross-Validation MSE: 0.5570677565691765
Test Set MSE: 0.7002544909535963
Random Forest Test Set MSE: 0.09982649138463481


/tmp/ipykernel_1262/591390400.py:44: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['Final_Score'].fillna(df['Score'], inplace=True)


### Display final results and export to Excel

In [7]:
# List of columns to be excluded
final_columns_exclude = ['Y/R', 'LG', 'ATT/game', 'YDS.1/game', 'TD.1/game']

# Exclude the specified columns from final_columns
final_columns = [col for col in final_columns if col not in final_columns_exclude]

# Create final analysis columns
analysis = df[['Rank', 'Final Rank', 'Player', 'Final_Score', 'Variance'] + final_columns]
analysis.set_index('Rank', inplace=True)
analysis = analysis.sort_values(by='Final Rank', ascending=True)

# Export to Excel
analysis.to_excel("WR_Analysis.xlsx", index=False)

# Display the top 30 rows
analysis.head(30)

,Final Rank,Player,Final_Score,Variance,G,FPTS,FPTS/G,REC/game,TGT/game,YDS/game,20+/game,TD/game
Rank,,,,,,,,,,,,
54,1,A.J. Brown (PHI),10.00,53,1,22.9,22.9,5.0,10.0,119.0,2.0,1.0
1,2,Malik Nabers (NYG),9.05,-1,3,68.5,22.8,7.7,12.3,90.3,2.0,1.0
2,3,Rashee Rice (KC),8.97,-1,3,64.9,21.6,8.0,9.7,96.0,1.0,0.7
5,4,Justin Jefferson (MIN),8.79,1,3,59.3,19.8,4.7,7.0,91.0,1.3,1.0
3,5,Chris Godwin (TB),8.58,-2,3,64.3,21.4,7.0,8.3,84.3,0.7,1.0
6,6,Nico Collins (HOU),8.37,0,3,57.8,19.3,6.0,9.3,112.7,1.7,0.3
7,7,DK Metcalf (SEA),8.14,0,3,55.2,18.4,5.7,8.0,87.3,1.0,0.7
19,8,Cooper Kupp (LAR),7.89,11,2,39.7,19.9,9.0,13.5,73.5,1.5,0.5
4,9,Jauan Jennings (SF),7.14,-5,3,63.6,21.2,6.0,7.0,92.0,1.7,1.0
